In [7]:
import re
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Fonction pour convertir les symboles d'actions
def convert_symbol(symbol: str) -> str:
    symbol_map = {
        "APPLE": "AAPL",
        "AMAZON": "AMZN",
        "MICROSOFT": "MSFT"
    }
    return symbol_map.get(symbol.upper(), symbol.upper())

# Fonction pour convertir une date au format SQL
def convert_date_format(phrase: str) -> str:
    date_pattern = r"\b(\d{1,2})/(\d{1,2})/(\d{4})\b"

    def replace_date(match):
        day, month, year = match.groups()
        try:
            # Convertir la date au format SQL (YYYY-MM-DD)
            date_obj = datetime.strptime(f"{day}/{month}/{year}", "%d/%m/%Y")
            return date_obj.strftime("%Y-%m-%d")
        except ValueError:
            return match.group(0)  # Retourner la date originale en cas d'erreur

    # Remplacer les dates dans la phrase
    return re.sub(date_pattern, replace_date, phrase)

# Fonction pour générer une requête SQL
def generer_requete_sql(description: str) -> str:
    """Génère une requête SQL basée sur une description en langage naturel."""
    
    # URL et modèle
    url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1"
    llm = ChatOpenAI(base_url=url, model="casperhansen/llama-3-70b-instruct-awq", api_key="nothing")

    # Prompt qui décrit la structure de la base de données et demande la requête SQL
    prompt_template = ChatPromptTemplate.from_template(
        """You are a database expert specializing in SQL queries. 
        Here is the structure of the database:

        - pdf_chunks: stores text data split into chunks.
        - stock_prices_finnhub: stores stock prices from Finnhub.
        - stock_prices_yahoo: stores daily stock prices from Yahoo Finance, with columns:
            - symbol (TEXT): stock symbol (e.g., 'AAPL', 'AMZN', 'MSFT')
            - date (DATE): date of the stock data
            - open_price (NUMERIC): opening price of the stock
            - high_price (NUMERIC): highest price of the stock
            - low_price (NUMERIC): lowest price of the stock
            - close_price (NUMERIC): closing price of the stock
            - volume (BIGINT): trading volume

        Generate a valid SQL query based on the given description.
        Use the 'stock_prices_yahoo' table for stock price queries.
        SQL output only (no extra text).

        Description: {description}
        SQL query:
        """
    )

    # Préparer et envoyer le prompt
    prompt = prompt_template.invoke({"description": description})
    response = llm.invoke(prompt).content

    try:
        # Vérifier que la réponse n'est pas vide
        if not response.strip():
            raise ValueError("Empty response from LLM.")

        return response.strip()

    except Exception as e:
        print(f"Error generating SQL query: {e}")
        return None


# Fonction principale pour traiter la description et ajuster la requête SQL
def generer_requete_sql_pour_phrase(phrase: str):
    """Génère une requête SQL, ajuste le symbole et les dates si nécessaire, et affiche le résultat."""
    
    # Identifier et convertir un symbole potentiel dans la phrase
    match = re.search(r"\b(Apple|Amazon|Microsoft)\b", phrase, re.IGNORECASE)
    
    if match:
        symbol = match.group(0)
        converted_symbol = convert_symbol(symbol)
        phrase = phrase.replace(symbol, converted_symbol)

    # Convertir les dates au format SQL dans la phrase
    phrase = convert_date_format(phrase)

    # Générer la requête SQL
    requete_sql = generer_requete_sql(phrase)

    if requete_sql:
        print("\nRequête SQL générée :")
        print(requete_sql)
    else:
        print("Erreur : Impossible de générer la requête SQL.")


# Exemple d'utilisation
phrase_exemple = "Quel a été le prix de l'action d'Apple le 11/12/2023 ?"
generer_requete_sql_pour_phrase(phrase_exemple)



Requête SQL générée :
SELECT close_price 
FROM stock_prices_yahoo 
WHERE symbol = 'AAPL' 
AND date = '2023-12-11';


In [5]:
pip install pandas requests langchain-core langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
phrase_exemple = "Quel était le cours de clôture de l'action Amazon le 11/12/2023 ?"
generer_requete_sql_pour_phrase(phrase_exemple)


Requête SQL générée :
SELECT close_price 
FROM stock_prices_yahoo 
WHERE symbol = 'AMZN' AND date = '2023-12-11';


In [10]:
phrase_exemple = "Donne-moi le prix de clôture de l'action Microsoft pour le 11 décembre 2023."
generer_requete_sql_pour_phrase(phrase_exemple)


Requête SQL générée :
SELECT close_price 
FROM stock_prices_yahoo 
WHERE symbol = 'MSFT' AND date = '2023-12-11';
